In [1]:
!python3 -m pip install --upgrade --user ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 6.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ortools
    Found existing installation: ortools 9.8.3296
    Uninstalling ortools-9.8.3296:
      Successfully uninstalled ortools-9.8.3296


In [32]:
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
from itertools import product, combinations
from collections import defaultdict
import pandas as pd
from pathlib import Path
from datetime import datetime
import numpy as np
from ast import literal_eval

In [2]:
excel_file = excel_file = Path("../data/ChallengeXHEC23022024.xlsx")

In [4]:
#  Generate combination for every 

In [6]:
df_caregivers = pd.read_excel(excel_file, sheet_name=2)
df_caregivers

,ID Intervenant,Latitude,Longitude,Compétences,Permis,Véhicule personnel,Dispo / Indispo
0,838320706,48.738516,1.391971,"AIDE MENAGERE, REPAS, TOILETTE",Oui,Oui,"Indispo 01/01, 30/01, 31/01"
1,609468992,48.640555,1.232581,"TOILETTE, REPAS, VIE SOCIALE, AIDE MENAGERE",Oui,Oui,"Indispo Tous les mercredis + 13/01, 14/01, 27/..."
2,78012267,48.729206,1.371985,"HOMMES TOUTES MAINS, JARDINAGE",Oui,Oui,Dispo le 25/01
3,818696864,48.744702,1.357921,"REPAS, AIDE MENAGERE, ACCOMPAGNEMENTS COURSES,...",Oui,Oui,Indispo tous les samedis et dimanche
4,746414886,48.769455,1.197644,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,Indispo tous les mercredis + le 05/01
5,78005437,48.712418,1.356278,"AIDE MENAGERE, ADMINISTRATION, REPAS, TOILETTE...",Oui,Oui,Indispo tous les samedis et dimanche
6,213237245,48.763226,1.241120,"TOILETTE, REPAS, AIDE MENAGERE",Oui,Oui,"Indispo 01/01, 02/01, 03/01, 13/01, 14/01, 16/..."
7,813991780,48.692715,1.073195,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Indispo tous les mercredis, vendredi, samedi, ..."
8,856089133,48.645418,1.523077,"ACCOMPAGNEMENTS COURSES PA, REPAS, TOILETTE, A...",Non,Non,"Indispo 05/01, 09/01, 11/01 + du 18/01 au 31/0..."
9,700168298,48.758099,1.210611,"TOILETTE, REPAS, AIDE MENAGERE, ACCOMPAGNEMENT...",Oui,Oui,"Dispo le 01/01, 30/01 et 31/01"


In [3]:
df_clients_commute = pd.read_csv(
                "../data/commute_driving_clients.csv"
            )
df_clients_commute["commute_minutes"] = (
    df_clients_commute["commute_seconds"] / 60
)
df_clients_commute

,clients,commute_seconds,commute_meters,source,destination,commute_method,commute_minutes
0,"(559475456, 559475456)",0,0,559475456,559475456,driving,0.000000
1,"(559475456, 559277088)",0,0,559475456,559277088,driving,0.000000
2,"(559475456, 87852633)",1070,13809,559475456,87852633,driving,17.833333
3,"(559475456, 243033408)",469,5880,559475456,243033408,driving,7.816667
4,"(559475456, 814940942)",326,2328,559475456,814940942,driving,5.433333
...,...,...,...,...,...,...,...
13919,"(1453084257, 334442311)",397,4232,1453084257,334442311,driving,6.616667
13920,"(1453084257, 621952854)",611,9328,1453084257,621952854,driving,10.183333
13921,"(1453084257, 1452858131)",952,13515,1453084257,1452858131,driving,15.866667
13922,"(1453084257, 846949801)",1015,12417,1453084257,846949801,driving,16.916667


In [8]:
df_care_clients_commute = pd.read_csv(
    "../data/commute_driving_care_clients.csv"
)
df_care_clients_commute["commute_minutes"] = (
    df_care_clients_commute["commute_seconds"] / 60
)
df_care_clients_commute

,care_client,commute_seconds,commute_meters,source,destination,commute_method,commute_minutes
0,"(838320706, 559475456)",326,3884,838320706,559475456,driving,5.433333
1,"(838320706, 559277088)",326,3884,838320706,559277088,driving,5.433333
2,"(838320706, 87852633)",965,15698,838320706,87852633,driving,16.083333
3,"(838320706, 243033408)",647,5489,838320706,243033408,driving,10.783333
4,"(838320706, 814940942)",275,2158,838320706,814940942,driving,4.583333
...,...,...,...,...,...,...,...
2827,"(776399834, 334442311)",847,10221,776399834,334442311,driving,14.116667
2828,"(776399834, 621952854)",360,2584,776399834,621952854,driving,6.000000
2829,"(776399834, 1452858131)",718,7530,776399834,1452858131,driving,11.966667
2830,"(776399834, 846949801)",521,3654,776399834,846949801,driving,8.683333


In [9]:
df_clients_care_commute = pd.read_csv(
    "../data/commute_driving_clients_care.csv"
)
df_clients_care_commute["commute_minutes"] = (
    df_clients_care_commute["commute_seconds"] / 60
)
df_clients_care_commute

,client_care,commute_seconds,commute_meters,source,destination,commute_method,commute_minutes
0,"(559475456, 838320706)",341,4084,559475456,838320706,driving,5.683333
1,"(559277088, 838320706)",341,4084,559277088,838320706,driving,5.683333
2,"(87852633, 838320706)",1000,13043,87852633,838320706,driving,16.666667
3,"(243033408, 838320706)",622,8444,243033408,838320706,driving,10.366667
4,"(814940942, 838320706)",336,2190,814940942,838320706,driving,5.600000
...,...,...,...,...,...,...,...
2827,"(334442311, 776399834)",856,10016,334442311,776399834,driving,14.266667
2828,"(621952854, 776399834)",386,2775,621952854,776399834,driving,6.433333
2829,"(1452858131, 776399834)",763,7634,1452858131,776399834,driving,12.716667
2830,"(846949801, 776399834)",560,3597,846949801,776399834,driving,9.333333


In [4]:
class CareScheduler:
    def __init__(self, excel_file):
        """
        Read cargiver and session data into Pandas DataFrames
        Args:
            excel_file (str): path to data in excel format
        """
        try:
            clients = pd.read_excel(excel_file, sheet_name=1)
            sessions = pd.read_excel(excel_file, sheet_name=0)

            # NOTE: remove later
            sessions = sessions.loc[sessions.Date == "2024-01-03", :]
            sessions["idx"] = sessions.index

            # create duration
            common_date = sessions["Date"].iloc[0]
            sessions["Heure de fin"] = sessions["Heure de fin"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Heure de début"] = sessions["Heure de début"].apply(
                lambda x: datetime.combine(common_date, x)
            )
            sessions["Duration"] = (
                sessions["Heure de fin"] - sessions["Heure de début"]
            )
            sessions["Duration"] = sessions["Duration"].apply(
                lambda x: x.seconds // 60
            )

            sessions["Start_time"] = sessions[
                "Heure de début"
            ] - pd.to_datetime("2024-01-03 00:00:00")
            sessions["Start_time"] = sessions["Start_time"].apply(
                lambda x: x.seconds // 60
            )

            # discard
            discard_list = [
                "ADMINISTRATION",
                "VISITE MEDICALE",
                "FORMATION",
                "COORDINATION",
                "HOMMES TOUTES MAINS",
            ]
            sessions = sessions[~sessions.Prestation.isin(discard_list)]

            sessions = sessions.merge(clients, how="left", on="ID Client")
            sessions = sessions.drop(columns="ID Intervenant")
            self.df_sessions = sessions
        except FileNotFoundError:
            print("Session data not found.")

        try:
            self.df_cargeivers = pd.read_excel(excel_file, sheet_name=2)
            # TODO: take kaans matrix -> filter cargeivers if they are not available today
        except FileNotFoundError:
            print("Caregiver data not found")

        try:
            df_clients_commute = pd.read_csv(
                "../data/commute_driving_clients.csv"
            )
            df_clients_commute["commute_minutes"] = (
                df_clients_commute["commute_seconds"] / 60
            )
            self.df_clients_commute = df_clients_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        try:
            df_care_clients_commute = pd.read_csv(
                "../data/commute_driving_care_clients.csv"
            )
            df_care_clients_commute["commute_minutes"] = (
                df_care_clients_commute["commute_seconds"] / 60
            )
            self.df_care_clients_commute = df_care_clients_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        try:
            df_clients_care_commute = pd.read_csv(
                "../data/commute_driving_clients_care.csv"
            )
            df_clients_care_commute["commute_minutes"] = (
                df_clients_care_commute["commute_seconds"] / 60
            )
            self.df_clients_care_commute = df_clients_care_commute
        except FileNotFoundError:
            print("Commute between clients data not found")

        # self.model = self.create_model()

    def _generate_case_durations(self):
        """
        Generate mapping of cases IDs to case time for the procedure
        Returns:
            (dict): dictionary with CaseID as key and median case time (mins) for procedure as value
        """
        return pd.Series(
            self.df_sessions["Duration"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_start_time(self, tasks):
        tasks_df = pd.DataFrame(tasks, columns=["idx", "Caregiver_ID"])

        temp = pd.DataFrame(
            self.df_sessions["Start_time"].values,
            index=self.df_sessions["idx"],
            columns=["Start_time"],
        )  # .to_dict()

        temp = tasks_df.merge(
            temp, how="left", right_index=True, left_on="idx"
        )

        arrays = [
            temp["idx"].to_list(),
            temp["Caregiver_ID"].to_list(),
        ]
        tuples = list(zip(*arrays))

        index = pd.MultiIndex.from_tuples(
            tuples, names=["idx", "Caregiver_ID"]
        )
        temp = pd.Series(temp["Start_time"].values, index=index)

        return temp.to_dict()

    def _generate_clients_commute(self):
        clients_commute = {}
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            try:
                clients_commute[(source, dest)] = self.df_clients_commute.loc[
                    (self.df_clients_commute.source == source)
                    & (self.df_clients_commute.destination == dest),
                    "commute_minutes",
                ].iloc[0]
            except IndexError:
                continue
        return clients_commute

    def _generate_care_clients_commute(self):
        care_clients_commute = {}
        for source, dest in product(
            self.df_cargeivers["ID Intervenant"].unique(),
            self.df_sessions["ID Client"].unique(),
        ):
            try:
                care_clients_commute[
                    (source, dest)
                ] = self.df_care_clients_commute.loc[
                    (self.df_care_clients_commute.source == source)
                    & (self.df_care_clients_commute.destination == dest),
                    "commute_minutes",
                ].iloc[
                    0
                ]
            except:
                continue
        return care_clients_commute

    def _generate_clients_care_commute(self):
        clients_care_commute = {}
        for source, dest in product(
            self.df_sessions["ID Client"].unique(),
            self.df_cargeivers["ID Intervenant"].unique(),
        ):
            try:
                clients_care_commute[
                    (source, dest)
                ] = self.df_clients_care_commute.loc[
                    (self.df_clients_care_commute.source == source)
                    & (self.df_clients_care_commute.destination == dest),
                    "commute_minutes",
                ].iloc[
                    0
                ]
            except:
                continue
        return clients_care_commute

    def _IDX_CLIENTS_match(self):
        return pd.Series(
            self.df_sessions["ID Client"].values, index=self.df_sessions["idx"]
        ).to_dict()

    def _generate_case_bigger(self):
        case_bigger = {}
        for case1, case2 in product(
            self.df_sessions["idx"].unique(), self.df_sessions["idx"].unique()
        ):
            case_bigger[(case1, case2)] = int(case1 > case2)
        return case_bigger

    def _generate_disjunctions(self):
        """Returns:
        disjunctions (list): list of tuples containing disjunctions
        """
        cases = self.df_sessions["idx"].to_list()
        cargivers = self.df_cargeivers["ID Intervenant"].to_list()
        disjunctions = []
        for (case1, case2, cargiver) in product(cases, cases, cargivers):
            if (case1 != case2) and (
                case2,
                case1,
                cargiver,
            ) not in disjunctions:
                disjunctions.append((case1, case2, cargiver))

        return disjunctions


In [5]:
problem = CareScheduler(excel_file)

In [76]:
disjunctions

[(177, 178, 838320706),
 (177, 178, 609468992),
 (177, 178, 78012267),
 (177, 178, 818696864),
 (177, 178, 746414886),
 (177, 178, 78005437),
 (177, 178, 213237245),
 (177, 178, 813991780),
 (177, 178, 856089133),
 (177, 178, 700168298),
 (177, 178, 810259688),
 (177, 178, 78007018),
 (177, 178, 764144581),
 (177, 178, 888888888),
 (177, 178, 854577575),
 (177, 178, 480302361),
 (177, 178, 78007874),
 (177, 178, 842621873),
 (177, 178, 710283561),
 (177, 178, 1452747150),
 (177, 178, 162858075),
 (177, 178, 739888851),
 (177, 178, 648993440),
 (177, 178, 776399834),
 (177, 179, 838320706),
 (177, 179, 609468992),
 (177, 179, 78012267),
 (177, 179, 818696864),
 (177, 179, 746414886),
 (177, 179, 78005437),
 (177, 179, 213237245),
 (177, 179, 813991780),
 (177, 179, 856089133),
 (177, 179, 700168298),
 (177, 179, 810259688),
 (177, 179, 78007018),
 (177, 179, 764144581),
 (177, 179, 888888888),
 (177, 179, 854577575),
 (177, 179, 480302361),
 (177, 179, 78007874),
 (177, 179, 842621873),

In [6]:
# For every session ID, we have the duration of cases
case_durations = problem._generate_case_durations()
case_durations

{177: 60,
 178: 45,
 179: 30,
 180: 45,
 181: 60,
 182: 30,
 183: 30,
 184: 45,
 185: 120,
 186: 60,
 187: 90,
 188: 75,
 189: 30,
 190: 30,
 191: 30,
 192: 30,
 193: 150,
 194: 30,
 195: 30,
 196: 90,
 197: 45,
 198: 30,
 199: 30,
 200: 30,
 201: 60,
 202: 30,
 203: 30,
 204: 45,
 205: 45,
 206: 60,
 207: 120,
 208: 45,
 209: 30,
 211: 60,
 212: 90,
 213: 60,
 214: 90,
 215: 60,
 216: 30,
 217: 45,
 218: 45,
 219: 60,
 220: 30,
 221: 60,
 222: 60,
 223: 30,
 224: 15,
 225: 60,
 226: 30,
 227: 45,
 228: 30,
 229: 30,
 230: 30,
 231: 30,
 232: 15,
 233: 120,
 234: 90,
 235: 120,
 236: 120,
 239: 180,
 240: 120,
 241: 120,
 242: 90,
 243: 75,
 244: 120,
 245: 90,
 246: 45,
 247: 15,
 248: 30,
 249: 45,
 250: 60,
 251: 30,
 252: 30,
 253: 30,
 254: 55,
 255: 30,
 256: 30,
 257: 30,
 258: 30,
 259: 15,
 260: 15,
 261: 60,
 262: 60,
 263: 45,
 264: 60,
 265: 30,
 266: 30}

In [7]:
tasks = list(product(problem.df_sessions["idx"], problem.df_cargeivers["ID Intervenant"]))

start_times = problem._generate_start_time(tasks)
start_times

{(177, 838320706): 450,
 (177, 609468992): 450,
 (177, 78012267): 450,
 (177, 818696864): 450,
 (177, 746414886): 450,
 (177, 78005437): 450,
 (177, 213237245): 450,
 (177, 813991780): 450,
 (177, 856089133): 450,
 (177, 700168298): 450,
 (177, 810259688): 450,
 (177, 78007018): 450,
 (177, 764144581): 450,
 (177, 888888888): 450,
 (177, 854577575): 450,
 (177, 480302361): 450,
 (177, 78007874): 450,
 (177, 842621873): 450,
 (177, 710283561): 450,
 (177, 1452747150): 450,
 (177, 162858075): 450,
 (177, 739888851): 450,
 (177, 648993440): 450,
 (177, 776399834): 450,
 (178, 838320706): 450,
 (178, 609468992): 450,
 (178, 78012267): 450,
 (178, 818696864): 450,
 (178, 746414886): 450,
 (178, 78005437): 450,
 (178, 213237245): 450,
 (178, 813991780): 450,
 (178, 856089133): 450,
 (178, 700168298): 450,
 (178, 810259688): 450,
 (178, 78007018): 450,
 (178, 764144581): 450,
 (178, 888888888): 450,
 (178, 854577575): 450,
 (178, 480302361): 450,
 (178, 78007874): 450,
 (178, 842621873): 450,

In [8]:
client_commutes = problem._generate_clients_commute()
client_commutes

{(522099746, 522099746): 0.0,
 (522099746, 87852633): 18.25,
 (522099746, 559277088): 3.6333333333333333,
 (522099746, 315244686): 3.0833333333333335,
 (522099746, 714782168): 4.55,
 (522099746, 861502972): 25.533333333333335,
 (522099746, 817127257): 3.1333333333333333,
 (522099746, 192451998): 8.133333333333333,
 (522099746, 433149947): 7.533333333333333,
 (522099746, 87274454): 15.366666666666667,
 (522099746, 78899980): 4.066666666666666,
 (522099746, 814940942): 2.3833333333333333,
 (522099746, 864411156): 25.533333333333335,
 (522099746, 803656603): 3.066666666666667,
 (522099746, 480337694): 8.0,
 (522099746, 712664807): 3.216666666666667,
 (522099746, 552306887): 0.18333333333333332,
 (522099746, 243033408): 8.116666666666667,
 (522099746, 860937824): 8.3,
 (522099746, 78867322): 3.5833333333333335,
 (522099746, 530138806): 7.633333333333334,
 (522099746, 794098228): 22.366666666666667,
 (522099746, 213156949): 8.566666666666666,
 (522099746, 446362468): 12.116666666666667,
 (5

In [9]:
care_client_commute = problem._generate_care_clients_commute()
care_client_commute

{(838320706, 522099746): 5.616666666666666,
 (838320706, 87852633): 16.083333333333332,
 (838320706, 559277088): 5.433333333333334,
 (838320706, 315244686): 5.766666666666667,
 (838320706, 714782168): 4.383333333333334,
 (838320706, 861502972): 23.033333333333335,
 (838320706, 817127257): 5.016666666666667,
 (838320706, 192451998): 10.8,
 (838320706, 433149947): 10.2,
 (838320706, 87274454): 14.433333333333334,
 (838320706, 78899980): 4.716666666666667,
 (838320706, 814940942): 4.583333333333333,
 (838320706, 864411156): 23.033333333333335,
 (838320706, 803656603): 4.95,
 (838320706, 480337694): 10.666666666666666,
 (838320706, 712664807): 4.9,
 (838320706, 552306887): 5.283333333333333,
 (838320706, 243033408): 10.783333333333333,
 (838320706, 860937824): 7.366666666666666,
 (838320706, 78867322): 5.383333333333334,
 (838320706, 530138806): 10.3,
 (838320706, 794098228): 19.866666666666667,
 (838320706, 213156949): 11.25,
 (838320706, 446362468): 9.533333333333333,
 (838320706, 466678

In [10]:
client_care_commute = problem._generate_clients_care_commute()

In [11]:
disjunctions = problem._generate_disjunctions()

In [72]:
len(disjunctions)

89784

- For each event, at most one caregiver can be assigned
- non-overlapping



In [13]:
model = cp_model.CpModel()

In [14]:
# boolean variables for the session
session_vars = {}
for session in problem.df_sessions["idx"]:
    for caregiver in problem.df_cargeivers["ID Intervenant"]:
        session_vars[(session, caregiver)] = model.NewBoolVar(f"session_{session}_caregiver_{caregiver}")
session_vars


{(177, 838320706): session_177_caregiver_838320706(0..1),
 (177, 609468992): session_177_caregiver_609468992(0..1),
 (177, 78012267): session_177_caregiver_78012267(0..1),
 (177, 818696864): session_177_caregiver_818696864(0..1),
 (177, 746414886): session_177_caregiver_746414886(0..1),
 (177, 78005437): session_177_caregiver_78005437(0..1),
 (177, 213237245): session_177_caregiver_213237245(0..1),
 (177, 813991780): session_177_caregiver_813991780(0..1),
 (177, 856089133): session_177_caregiver_856089133(0..1),
 (177, 700168298): session_177_caregiver_700168298(0..1),
 (177, 810259688): session_177_caregiver_810259688(0..1),
 (177, 78007018): session_177_caregiver_78007018(0..1),
 (177, 764144581): session_177_caregiver_764144581(0..1),
 (177, 888888888): session_177_caregiver_888888888(0..1),
 (177, 854577575): session_177_caregiver_854577575(0..1),
 (177, 480302361): session_177_caregiver_480302361(0..1),
 (177, 78007874): session_177_caregiver_78007874(0..1),
 (177, 842621873): ses

In [15]:
# For each session at most one
for s in problem.df_sessions["idx"]:
    model.AddExactlyOne(session_vars[(s, n)] for n in problem.df_cargeivers["ID Intervenant"])

In [83]:
problem.df_sessions

,ID Client,Date,Heure de début,Heure de fin,Prestation,idx,Duration,Start_time,Latitude,Longitude
0,522099746,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:30:00,TOILETTE,177,60,450,48.729059,1.374504
1,87852633,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:15:00,TOILETTE,178,45,450,48.691944,1.498773
2,559277088,2024-01-03,2024-01-03 07:45:00,2024-01-03 08:15:00,TOILETTE,179,30,465,48.721052,1.375689
3,315244686,2024-01-03,2024-01-03 08:00:00,2024-01-03 08:45:00,TOILETTE,180,45,480,48.732218,1.363744
4,714782168,2024-01-03,2024-01-03 08:00:00,2024-01-03 09:00:00,TOILETTE,181,60,480,48.740158,1.369426
...,...,...,...,...,...,...,...,...,...,...
82,794098228,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,262,60,1135,48.764898,1.203185
83,559475456,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:40:00,TOILETTE,263,45,1135,48.721052,1.375689
84,714782168,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,264,60,1135,48.740158,1.369426
85,803656603,2024-01-03,2024-01-03 19:50:00,2024-01-03 20:20:00,TOILETTE,265,30,1190,48.723600,1.382000


In [16]:
from itertools import combinations
# combinations(problem.df_clients_commute)
problem.df_clients_commute

,clients,commute_seconds,commute_meters,source,destination,commute_method,commute_minutes
0,"(559475456, 559475456)",0,0,559475456,559475456,driving,0.000000
1,"(559475456, 559277088)",0,0,559475456,559277088,driving,0.000000
2,"(559475456, 87852633)",1070,13809,559475456,87852633,driving,17.833333
3,"(559475456, 243033408)",469,5880,559475456,243033408,driving,7.816667
4,"(559475456, 814940942)",326,2328,559475456,814940942,driving,5.433333
...,...,...,...,...,...,...,...
13919,"(1453084257, 334442311)",397,4232,1453084257,334442311,driving,6.616667
13920,"(1453084257, 621952854)",611,9328,1453084257,621952854,driving,10.183333
13921,"(1453084257, 1452858131)",952,13515,1453084257,1452858131,driving,15.866667
13922,"(1453084257, 846949801)",1015,12417,1453084257,846949801,driving,16.916667


In [24]:
session1 = 177
session2 = 178
problem.df_clients_commute[(problem.df_clients_commute.source==session_client_mapping[session1])&(problem.df_clients_commute.destination==session_client_mapping[session2])]["commute_minutes"].iloc[0]

18.25

In [25]:
for (i1, i2) in problem.df_clients_commute["clients"].apply(literal_eval):
    print(i1, i2)
    break

559475456 559475456


In [126]:
problem.df_clients_commute

,clients,commute_seconds,commute_meters,source,destination,commute_method,commute_minutes
0,"(559475456, 559277088)",0,0,559475456,559277088,driving,0.000000
1,"(559475456, 87852633)",1070,13809,559475456,87852633,driving,17.833333
2,"(559475456, 243033408)",469,5880,559475456,243033408,driving,7.816667
3,"(559475456, 814940942)",326,2328,559475456,814940942,driving,5.433333
4,"(559475456, 864411156)",1511,19690,559475456,864411156,driving,25.183333
...,...,...,...,...,...,...,...
6898,"(621952854, 846949801)",518,3404,621952854,846949801,driving,8.633333
6899,"(621952854, 1453084257)",634,9622,621952854,1453084257,driving,10.566667
6900,"(1452858131, 846949801)",618,7602,1452858131,846949801,driving,10.300000
6901,"(1452858131, 1453084257)",882,13232,1452858131,1453084257,driving,14.700000


In [19]:
# Distance between every session
# Find the corresponding client per session, using the mapping, find the clients and then find the commute time
client_combos = combinations(problem.df_sessions["ID Client"].unique(), 2)
session_client_mapping = problem.df_sessions.set_index("idx")["ID Client"].to_dict()

In [30]:
problem.df_sessions

,ID Client,Date,Heure de début,Heure de fin,Prestation,idx,Duration,Start_time,Latitude,Longitude
0,522099746,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:30:00,TOILETTE,177,60,450,48.729059,1.374504
1,87852633,2024-01-03,2024-01-03 07:30:00,2024-01-03 08:15:00,TOILETTE,178,45,450,48.691944,1.498773
2,559277088,2024-01-03,2024-01-03 07:45:00,2024-01-03 08:15:00,TOILETTE,179,30,465,48.721052,1.375689
3,315244686,2024-01-03,2024-01-03 08:00:00,2024-01-03 08:45:00,TOILETTE,180,45,480,48.732218,1.363744
4,714782168,2024-01-03,2024-01-03 08:00:00,2024-01-03 09:00:00,TOILETTE,181,60,480,48.740158,1.369426
...,...,...,...,...,...,...,...,...,...,...
82,794098228,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,262,60,1135,48.764898,1.203185
83,559475456,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:40:00,TOILETTE,263,45,1135,48.721052,1.375689
84,714782168,2024-01-03,2024-01-03 18:55:00,2024-01-03 19:55:00,REPAS,264,60,1135,48.740158,1.369426
85,803656603,2024-01-03,2024-01-03 19:50:00,2024-01-03 20:20:00,TOILETTE,265,30,1190,48.723600,1.382000


In [31]:
def test_overlap(session1, session2):
    commute_time = (problem.df_clients_commute[
                    (problem.df_clients_commute.source==session_client_mapping[session1])&
                    (problem.df_clients_commute.destination==session_client_mapping[session2])]
                    ["commute_minutes"].iloc[0]
                    )
    if problem.df_sessions[problem.df_sessions["idx"]==session1]["Start_time"].iloc[0] + commute_time > \
        problem.df_sessions[problem.df_sessions["idx"]==session2]["Start_time"].iloc[0]:
        return True
    return False
    
for (r1, r2) in combinations(problem.df_sessions["idx"], 2):
    if test_overlap(r1,r2):
            for caregiver in problem.df_cargeivers["ID Intervenant"]:
                model.AddBoolOr([session_vars[r1, caregiver].Not(), session_vars[r2, caregiver].Not()])

In [35]:
session_vars

{(177, 838320706): session_177_caregiver_838320706(0..1),
 (177, 609468992): session_177_caregiver_609468992(0..1),
 (177, 78012267): session_177_caregiver_78012267(0..1),
 (177, 818696864): session_177_caregiver_818696864(0..1),
 (177, 746414886): session_177_caregiver_746414886(0..1),
 (177, 78005437): session_177_caregiver_78005437(0..1),
 (177, 213237245): session_177_caregiver_213237245(0..1),
 (177, 813991780): session_177_caregiver_813991780(0..1),
 (177, 856089133): session_177_caregiver_856089133(0..1),
 (177, 700168298): session_177_caregiver_700168298(0..1),
 (177, 810259688): session_177_caregiver_810259688(0..1),
 (177, 78007018): session_177_caregiver_78007018(0..1),
 (177, 764144581): session_177_caregiver_764144581(0..1),
 (177, 888888888): session_177_caregiver_888888888(0..1),
 (177, 854577575): session_177_caregiver_854577575(0..1),
 (177, 480302361): session_177_caregiver_480302361(0..1),
 (177, 78007874): session_177_caregiver_78007874(0..1),
 (177, 842621873): ses

In [33]:
cases_assigned = defaultdict(list)
for caregiver in problem.df_cargeivers["ID Intervenant"]:
    for session in problem.df_sessions["idx"]:
      if session_vars[session, caregiver] == 1:
            cases_assigned[caregiver] += [session]

commute_times = defaultdict(int)
for caregiver, cases in cases_assigned.items():
      i = 0 
      while i < len(cases) - 1:
            commute_times[caregiver] += commute[cases[i], cases[i+1]]



NotImplementedError: Evaluating a BoundedLinearExpression "session_177_caregiver_838320706 == 1" as a Boolean value is not supported.

In [34]:
model.Minimize(sum(commute_times.values()))

NameError: name 'commute_times' is not defined

# EXAMPLE

In [1]:
#!/usr/bin/env python3
# Copyright 2010-2024 Google LLC
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Creates a shift scheduling problem and solves it."""

from absl import app
from absl import flags

from google.protobuf import text_format
from ortools.sat.python import cp_model

_OUTPUT_PROTO = flags.DEFINE_string(
    "output_proto", "", "Output file to write the cp_model proto to."
)
_PARAMS = flags.DEFINE_string(
    "params", "max_time_in_seconds:10.0", "Sat solver parameters."
)


def negated_bounded_span(
    works: list, start: int, length: int
) -> list:
    """Filters an isolated sub-sequence of variables assined to True.

    Extract the span of Boolean variables [start, start + length), negate them,
    and if there is variables to the left/right of this span, surround the span by
    them in non negated form.

    Args:
      works: a list of variables to extract the span from.
      start: the start to the span.
      length: the length of the span.

    Returns:
      a list of variables which conjunction will be false if the sub-list is
      assigned to True, and correctly bounded by variables assigned to False,
      or by the start or end of works.
    """
    sequence = []
    # left border (start of works, or works[start - 1])
    if start > 0:
        sequence.append(works[start - 1])
    for i in range(length):
        sequence.append(~works[start + i])
    # right border (end of works or works[start + length])
    if start + length < len(works):
        sequence.append(works[start + length])
    return sequence


def add_soft_sequence_constraint(
    model: cp_model.CpModel,
    works: list,
    hard_min: int,
    soft_min: int,
    min_cost: int,
    soft_max: int,
    hard_max: int,
    max_cost: int,
    prefix: str,
) -> tuple[list, list[int]]:
    """Sequence constraint on true variables with soft and hard bounds.

    This constraint look at every maximal contiguous sequence of variables
    assigned to true. If forbids sequence of length < hard_min or > hard_max.
    Then it creates penalty terms if the length is < soft_min or > soft_max.

    Args:
      model: the sequence constraint is built on this model.
      works: a list of Boolean variables.
      hard_min: any sequence of true variables must have a length of at least
        hard_min.
      soft_min: any sequence should have a length of at least soft_min, or a
        linear penalty on the delta will be added to the objective.
      min_cost: the coefficient of the linear penalty if the length is less than
        soft_min.
      soft_max: any sequence should have a length of at most soft_max, or a linear
        penalty on the delta will be added to the objective.
      hard_max: any sequence of true variables must have a length of at most
        hard_max.
      max_cost: the coefficient of the linear penalty if the length is more than
        soft_max.
      prefix: a base name for penalty literals.

    Returns:
      a tuple (variables_list, coefficient_list) containing the different
      penalties created by the sequence constraint.
    """
    cost_literals = []
    cost_coefficients = []

    # Forbid sequences that are too short.
    for length in range(1, hard_min):
        for start in range(len(works) - length + 1):
            model.add_bool_or(negated_bounded_span(works, start, length))

    # Penalize sequences that are below the soft limit.
    if min_cost > 0:
        for length in range(hard_min, soft_min):
            for start in range(len(works) - length + 1):
                span = negated_bounded_span(works, start, length)
                name = ": under_span(start=%i, length=%i)" % (start, length)
                lit = model.new_bool_var(prefix + name)
                span.append(lit)
                model.add_bool_or(span)
                cost_literals.append(lit)
                # We filter exactly the sequence with a short length.
                # The penalty is proportional to the delta with soft_min.
                cost_coefficients.append(min_cost * (soft_min - length))

    # Penalize sequences that are above the soft limit.
    if max_cost > 0:
        for length in range(soft_max + 1, hard_max + 1):
            for start in range(len(works) - length + 1):
                span = negated_bounded_span(works, start, length)
                name = ": over_span(start=%i, length=%i)" % (start, length)
                lit = model.new_bool_var(prefix + name)
                span.append(lit)
                model.add_bool_or(span)
                cost_literals.append(lit)
                # Cost paid is max_cost * excess length.
                cost_coefficients.append(max_cost * (length - soft_max))

    # Just forbid any sequence of true variables with length hard_max + 1
    for start in range(len(works) - hard_max):
        model.add_bool_or([~works[i] for i in range(start, start + hard_max + 1)])
    return cost_literals, cost_coefficients


def add_soft_sum_constraint(
    model: cp_model.CpModel,
    works: list,
    hard_min: int,
    soft_min: int,
    min_cost: int,
    soft_max: int,
    hard_max: int,
    max_cost: int,
    prefix: str,
) -> tuple[list[cp_model.IntVar], list[int]]:
    """sum constraint with soft and hard bounds.

    This constraint counts the variables assigned to true from works.
    If forbids sum < hard_min or > hard_max.
    Then it creates penalty terms if the sum is < soft_min or > soft_max.

    Args:
      model: the sequence constraint is built on this model.
      works: a list of Boolean variables.
      hard_min: any sequence of true variables must have a sum of at least
        hard_min.
      soft_min: any sequence should have a sum of at least soft_min, or a linear
        penalty on the delta will be added to the objective.
      min_cost: the coefficient of the linear penalty if the sum is less than
        soft_min.
      soft_max: any sequence should have a sum of at most soft_max, or a linear
        penalty on the delta will be added to the objective.
      hard_max: any sequence of true variables must have a sum of at most
        hard_max.
      max_cost: the coefficient of the linear penalty if the sum is more than
        soft_max.
      prefix: a base name for penalty variables.

    Returns:
      a tuple (variables_list, coefficient_list) containing the different
      penalties created by the sequence constraint.
    """
    cost_variables = []
    cost_coefficients = []
    sum_var = model.new_int_var(hard_min, hard_max, "")
    # This adds the hard constraints on the sum.
    model.add(sum_var == sum(works))

    # Penalize sums below the soft_min target.
    if soft_min > hard_min and min_cost > 0:
        delta = model.new_int_var(-len(works), len(works), "")
        model.add(delta == soft_min - sum_var)
        # TODO(user): Compare efficiency with only excess >= soft_min - sum_var.
        excess = model.new_int_var(0, 7, prefix + ": under_sum")
        model.add_max_equality(excess, [delta, 0])
        cost_variables.append(excess)
        cost_coefficients.append(min_cost)

    # Penalize sums above the soft_max target.
    if soft_max < hard_max and max_cost > 0:
        delta = model.new_int_var(-7, 7, "")
        model.add(delta == sum_var - soft_max)
        excess = model.new_int_var(0, 7, prefix + ": over_sum")
        model.add_max_equality(excess, [delta, 0])
        cost_variables.append(excess)
        cost_coefficients.append(max_cost)

    return cost_variables, cost_coefficients


def solve_shift_scheduling(params: str, output_proto: str):
    """Solves the shift scheduling problem."""
    # Data
    num_employees = 8
    num_weeks = 3
    shifts = ["O", "M", "A", "N"]

    # Fixed assignment: (employee, shift, day).
    # This fixes the first 2 days of the schedule.
    fixed_assignments = [
        (0, 0, 0),
        (1, 0, 0),
        (2, 1, 0),
        (3, 1, 0),
        (4, 2, 0),
        (5, 2, 0),
        (6, 2, 3),
        (7, 3, 0),
        (0, 1, 1),
        (1, 1, 1),
        (2, 2, 1),
        (3, 2, 1),
        (4, 2, 1),
        (5, 0, 1),
        (6, 0, 1),
        (7, 3, 1),
    ]

    # Request: (employee, shift, day, weight)
    # A negative weight indicates that the employee desire this assignment.
    requests = [
        # Employee 3 does not want to work on the first Saturday (negative weight
        # for the Off shift).
        (3, 0, 5, -2),
        # Employee 5 wants a night shift on the second Thursday (negative weight).
        (5, 3, 10, -2),
        # Employee 2 does not want a night shift on the first Friday (positive
        # weight).
        (2, 3, 4, 4),
    ]

    # Shift constraints on continuous sequence :
    #     (shift, hard_min, soft_min, min_penalty,
    #             soft_max, hard_max, max_penalty)
    shift_constraints = [
        # One or two consecutive days of rest, this is a hard constraint.
        (0, 1, 1, 0, 2, 2, 0),
        # between 2 and 3 consecutive days of night shifts, 1 and 4 are
        # possible but penalized.
        (3, 1, 2, 20, 3, 4, 5),
    ]

    # Weekly sum constraints on shifts days:
    #     (shift, hard_min, soft_min, min_penalty,
    #             soft_max, hard_max, max_penalty)
    weekly_sum_constraints = [
        # Constraints on rests per week.
        (0, 1, 2, 7, 2, 3, 4),
        # At least 1 night shift per week (penalized). At most 4 (hard).
        (3, 0, 1, 3, 4, 4, 0),
    ]

    # Penalized transitions:
    #     (previous_shift, next_shift, penalty (0 means forbidden))
    penalized_transitions = [
        # Afternoon to night has a penalty of 4.
        (2, 3, 4),
        # Night to morning is forbidden.
        (3, 1, 0),
    ]

    # daily demands for work shifts (morning, afternon, night) for each day
    # of the week starting on Monday.
    weekly_cover_demands = [
        (2, 3, 1),  # Monday
        (2, 3, 1),  # Tuesday
        (2, 2, 2),  # Wednesday
        (2, 3, 1),  # Thursday
        (2, 2, 2),  # Friday
        (1, 2, 3),  # Saturday
        (1, 3, 1),  # Sunday
    ]

    # Penalty for exceeding the cover constraint per shift type.
    excess_cover_penalties = (2, 2, 5)

    num_days = num_weeks * 7
    num_shifts = len(shifts)

    model = cp_model.CpModel()

    work = {}
    for e in range(num_employees):
        for s in range(num_shifts):
            for d in range(num_days):
                work[e, s, d] = model.new_bool_var("work%i_%i_%i" % (e, s, d))

    # Linear terms of the objective in a minimization context.
    obj_int_vars: list[cp_model.IntVar] = []
    obj_int_coeffs: list[int] = []
    obj_bool_vars: list = []
    obj_bool_coeffs: list[int] = []

    # Exactly one shift per day.
    for e in range(num_employees):
        for d in range(num_days):
            model.add_exactly_one(work[e, s, d] for s in range(num_shifts))

    # Fixed assignments.
    for e, s, d in fixed_assignments:
        model.add(work[e, s, d] == 1)

    # Employee requests
    for e, s, d, w in requests:
        obj_bool_vars.append(work[e, s, d])
        obj_bool_coeffs.append(w)

    # Shift constraints
    for ct in shift_constraints:
        shift, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
        for e in range(num_employees):
            works = [work[e, shift, d] for d in range(num_days)]
            variables, coeffs = add_soft_sequence_constraint(
                model,
                works,
                hard_min,
                soft_min,
                min_cost,
                soft_max,
                hard_max,
                max_cost,
                "shift_constraint(employee %i, shift %i)" % (e, shift),
            )
            obj_bool_vars.extend(variables)
            obj_bool_coeffs.extend(coeffs)

    # Weekly sum constraints
    for ct in weekly_sum_constraints:
        shift, hard_min, soft_min, min_cost, soft_max, hard_max, max_cost = ct
        for e in range(num_employees):
            for w in range(num_weeks):
                works = [work[e, shift, d + w * 7] for d in range(7)]
                variables, coeffs = add_soft_sum_constraint(
                    model,
                    works,
                    hard_min,
                    soft_min,
                    min_cost,
                    soft_max,
                    hard_max,
                    max_cost,
                    "weekly_sum_constraint(employee %i, shift %i, week %i)"
                    % (e, shift, w),
                )
                obj_int_vars.extend(variables)
                obj_int_coeffs.extend(coeffs)

    # Penalized transitions
    for previous_shift, next_shift, cost in penalized_transitions:
        for e in range(num_employees):
            for d in range(num_days - 1):
                transition = [
                    ~work[e, previous_shift, d],
                    ~work[e, next_shift, d + 1],
                ]
                if cost == 0:
                    model.add_bool_or(transition)
                else:
                    trans_var = model.new_bool_var(
                        "transition (employee=%i, day=%i)" % (e, d)
                    )
                    transition.append(trans_var)
                    model.add_bool_or(transition)
                    obj_bool_vars.append(trans_var)
                    obj_bool_coeffs.append(cost)

    # Cover constraints
    for s in range(1, num_shifts):
        for w in range(num_weeks):
            for d in range(7):
                works = [work[e, s, w * 7 + d] for e in range(num_employees)]
                # Ignore Off shift.
                min_demand = weekly_cover_demands[d][s - 1]
                worked = model.new_int_var(min_demand, num_employees, "")
                model.add(worked == sum(works))
                over_penalty = excess_cover_penalties[s - 1]
                if over_penalty > 0:
                    name = "excess_demand(shift=%i, week=%i, day=%i)" % (s, w, d)
                    excess = model.new_int_var(0, num_employees - min_demand, name)
                    model.add(excess == worked - min_demand)
                    obj_int_vars.append(excess)
                    obj_int_coeffs.append(over_penalty)

    # Objective
    model.minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i] for i in range(len(obj_bool_vars)))
        + sum(obj_int_vars[i] * obj_int_coeffs[i] for i in range(len(obj_int_vars)))
    )

    if output_proto:
        print("Writing proto to %s" % output_proto)
        with open(output_proto, "w") as text_file:
            text_file.write(str(model))

    # Solve the model.
    solver = cp_model.CpSolver()
    if params:
        text_format.Parse(params, solver.parameters)
    solution_printer = cp_model.ObjectiveSolutionPrinter()
    status = solver.solve(model, solution_printer)

    # Print solution.
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        print()
        header = "          "
        for w in range(num_weeks):
            header += "M T W T F S S "
        print(header)
        for e in range(num_employees):
            schedule = ""
            for d in range(num_days):
                for s in range(num_shifts):
                    if solver.boolean_value(work[e, s, d]):
                        schedule += shifts[s] + " "
            print("worker %i: %s" % (e, schedule))
        print()
        print("Penalties:")
        for i, var in enumerate(obj_bool_vars):
            if solver.boolean_value(var):
                penalty = obj_bool_coeffs[i]
                if penalty > 0:
                    print(f"  {var.name} violated, penalty={penalty}")
                else:
                    print(f"  {var.name} fulfilled, gain={-penalty}")

        for i, var in enumerate(obj_int_vars):
            if solver.value(var) > 0:
                print(
                    "  %s violated by %i, linear penalty=%i"
                    % (var.name, solver.value(var), obj_int_coeffs[i])
                )

    print()
    print("Statistics")
    print("  - status          : %s" % solver.status_name(status))
    print("  - conflicts       : %i" % solver.num_conflicts)
    print("  - branches        : %i" % solver.num_branches)
    print("  - wall time       : %f s" % solver.wall_time)



solve_shift_scheduling(_PARAMS.value, _OUTPUT_PROTO.value)


UnparsedFlagAccessError: Trying to access flag --params before flags were parsed.

TypeError: int() argument must be a string, a bytes-like object or a number, not 'FlagHolder'